In [24]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [25]:
def print_2d_matrix(matrix, decimals=3):
    for row in matrix:
        print(" ".join(f"{val:9.{decimals}f}" for val in row))

In [26]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [27]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# WFN data
wfndata = wfn2py(dirname+'WFN.h5')
wfnqdata = wfn2py(dirname+'WFNq.h5')

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

In [28]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 10001.32it/s]


In [29]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 138.34it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [30]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)

kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    q0=q0,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [31]:
numq = kernelclass.qpts.numq
numk = kernelclass.kpts.numk

In [32]:
ikf, ikc = np.random.randint(0, numk, size=2)
print(f"ikf, ikc = {ikf}, {ikc}")

ikf, ikc = 6, 6


In [33]:
from intkernel import InterpMtxEl

InterpClass = InterpMtxEl.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=kernelclass,
    parallel=True,
)

In [34]:
coeff_val = InterpClass.coeff_mtxel(ikf, ikf, "val")
coeff_con = InterpClass.coeff_mtxel(ikf, ikc, "con")


print(f"coeff_val is")
print_2d_matrix(coeff_val, decimals=3)

print(f"\n coeff_con is")
print_2d_matrix(coeff_con, decimals=3)

coeff_val is
1.000+0.000j 0.000-0.000j -0.000+0.000j -0.000-0.000j
0.000+0.000j 1.000+0.000j 0.000-0.000j -0.000-0.000j
-0.000-0.000j 0.000+0.000j 1.000-0.000j 0.000+0.000j
-0.000+0.000j -0.000+0.000j 0.000-0.000j 1.000+0.000j

 coeff_con is
1.000-0.000j -0.000+0.000j 0.000-0.000j 0.000-0.000j
-0.000-0.000j 1.000-0.000j 0.000-0.000j 0.000-0.000j
0.000+0.000j 0.000+0.000j 1.000+0.000j 0.000-0.000j
0.000+0.000j 0.000+0.000j 0.000+0.000j 1.000-0.000j
